In [133]:
# Cassandra database

In [150]:
from cassandra.cluster import Cluster
from cassandra.query import dict_factory
from cassandra.query import SimpleStatement
import uuid
import sqlite3
import time
import random
from datetime import datetime, timedelta

In [163]:
uuid.uuid4()

UUID('9f2ff6d3-f373-44f7-916a-0756afc27779')

In [164]:
cluster = Cluster(["127.0.0.1"],port=9042)

In [165]:
session = cluster.connect()

In [166]:
create_keyspace_query = """
CREATE KEYSPACE IF NOT EXISTS ecommerce
WITH replication = {'class': 'SimpleStrategy', 'replication_factor': 1}
"""
session.execute(create_keyspace_query)

In [167]:
# Data Model Creation:
# Design a data model for an e-commerce platform to handle products, orders, and user information.
# Define appropriate column families and primary keys to ensure efficient querying.
# Insert sample data into the Cassandra database, including user information and product details.

In [168]:
session.set_keyspace('ecommerce')

In [169]:
# Create products table
create_products_table_query = """
CREATE TABLE IF NOT EXISTS products (
    product_id UUID PRIMARY KEY,
    name TEXT,
    description TEXT,
    price DECIMAL,
    category TEXT
)
"""
session.execute(create_products_table_query)

In [114]:
# insert product data
insert_product_query = """
INSERT INTO products (product_id, name, description, price, category)
VALUES (%(product_id)s, %(name)s, %(description)s, %(price)s, %(category)s)
"""

products_data = [
    {"product_id": uuid.UUID("3dca3e67-1b43-4c5e-9cfd-9423b4878f01"), "name": "Laptop", "description": "High-performance laptop", "price": 1200, "category": "Electronics"},
    {"product_id": uuid.UUID("4eab2b59-4c2f-4ec0-a865-86dbca73e869"), "name": "Monitor", "description": "27-inch 4K monitor", "price": 300, "category": "Electronics"},
    {"product_id": uuid.UUID("3e6fa09d-2a67-4f08-86e3-4efc5c42c22a"), "name": "Keyboard", "description": "logitech k120", "price": 40, "category": "Electronics"}
]

for product in products_data:
    session.execute(insert_product_query, product)


In [110]:
# Create orders table
create_orders_table_query = """
CREATE TABLE IF NOT EXISTS orders (
    order_id UUID PRIMARY KEY,
    user_id UUID,
    order_date TIMESTAMP,
    total_amount DECIMAL,
    status TEXT
)
"""
session.execute(create_orders_table_query)

In [117]:
# insert orders data
insert_order_query = """
INSERT INTO orders (order_id, user_id, order_date, total_amount, status)
VALUES (%(order_id)s, %(user_id)s, %(order_date)s, %(total_amount)s, %(status)s)
"""

orders_data = [
    {"order_id": uuid.UUID("5e563a2b-79c2-4b66-9ca2-1f3ca812e1a2"), "user_id": uuid.UUID("1f14b2e1-4857-4c98-882d-8d2e818e6b68"), "order_date": datetime.now(), "total_amount": 15, "status": "Completed"},
    {"order_id": uuid.UUID("6f84729e-50d7-48b7-91c0-2ec55feab89d"), "user_id": uuid.UUID("1f14b2e1-4857-4c98-882d-8d2e818e6b68"), "order_date": datetime.now(), "total_amount": 3, "status": "Pending"},
    {"order_id": uuid.UUID("7d91444a-2c35-4c22-842a-99e3f6ce8c4b"), "user_id": uuid.UUID("2f84b82f-9e7e-4d4c-88e9-3b40d1d10d4a"), "order_date": datetime.now(), "total_amount": 9, "status": "Completed"}
]

for order in orders_data:
    session.execute(insert_order_query, order)

In [118]:
# Create users table
create_users_table_query = """
CREATE TABLE IF NOT EXISTS users (
    user_id UUID PRIMARY KEY,
    name TEXT,
    email TEXT,
    address TEXT
)
"""
session.execute(create_users_table_query)

In [119]:
# insert users data
insert_user_query = """
INSERT INTO users (user_id, name, email, address)
VALUES (%(user_id)s, %(name)s, %(email)s, %(address)s)
"""
users_data = [
    {"user_id": uuid.UUID("1f14b2e1-4857-4c98-882d-8d2e818e6b68"), "name": "Karlis Zarins", "email": "karlis.zarins@gmail.com", "address": "Meistaru 6, Riga"},
    {"user_id": uuid.UUID("2f84b82f-9e7e-4d4c-88e9-3b40d1d10d4a"), "name": "Zane Caune", "email": "Zane.caune@gmail.com", "address": "Bikernieku 7, Riga"},
    {"user_id": uuid.UUID("a2bfb965-68e6-4d8c-b181-9c6a2a3c4b50"), "name": "Edgars Raunis", "email": "Edgars.raunis@gmail.com", "address": "Maskavas 250/2, Riga"}
]
for user in users_data:
    session.execute(insert_user_query, user)

In [125]:
# Retrieve a user's order history using CQL (Cassandra Query Language).

session.row_factory = dict_factory
user_id = uuid.UUID("1f14b2e1-4857-4c98-882d-8d2e818e6b68")

select_orders_query = """
SELECT * FROM orders WHERE user_id = %(user_id)s ALLOW FILTERING
"""

order_history = session.execute(select_orders_query, {"user_id": user_id})

for order in order_history:
    print("Order ID:", order["order_id"])
    print("Order Date:", order["order_date"])
    print("Total Amount:", order["total_amount"])
    print("Status:", order["status"])

Order ID: 5e563a2b-79c2-4b66-9ca2-1f3ca812e1a2
Order Date: 2023-08-13 10:13:57.156000
Total Amount: 15
Status: Completed
Order ID: 6f84729e-50d7-48b7-91c0-2ec55feab89d
Order Date: 2023-08-13 10:13:57.156000
Total Amount: 3
Status: Pending


In [126]:
# Time-Series Data:
# Design a schema to handle time-series data, such as IoT sensor readings.
# Insert and retrieve time-series data efficiently, using appropriate time-based partitioning.

In [159]:
# Design the schema with time-based data
create_sensor_readings_table_query = """
CREATE TABLE IF NOT EXISTS sensor_readings (
    warehouse_id UUID,
    timestamp TIMESTAMP,
    temperature FLOAT,
    PRIMARY KEY (warehouse_id, timestamp)
) WITH CLUSTERING ORDER BY (timestamp DESC);
"""

session.execute(create_sensor_readings_table_query)

In [170]:
# Insert sensor readings data with simulated time series
warehouse_id = uuid.uuid4()

for _ in range(100):
    timestamp = datetime.now()
    temperature = 20.0 + random.uniform(-5.0, 5.0)
    insert_reading_query = """
    INSERT INTO sensor_readings (warehouse_id, timestamp, temperature)
    VALUES (%s, %s, %s)
    """
    session.execute(insert_reading_query, (warehouse_id, timestamp, temperature))

In [171]:
# Retrieve sensor readings data 
start_time = datetime.now() - timedelta(minutes=30)
end_time = datetime.now()

select_readings_query = """
SELECT * FROM sensor_readings
WHERE warehouse_id = %s AND timestamp >= %s AND timestamp <= %s
"""

result_set = session.execute(select_readings_query, (warehouse_id, start_time, end_time))

for row in result_set:
    print(row)

session.shutdown()
cluster.shutdown()

Row(warehouse_id=UUID('8bfed691-e001-402c-97aa-a1e972812ccb'), timestamp=datetime.datetime(2023, 8, 13, 10, 57, 8, 9000), temperature=18.845659255981445)
Row(warehouse_id=UUID('8bfed691-e001-402c-97aa-a1e972812ccb'), timestamp=datetime.datetime(2023, 8, 13, 10, 57, 7, 994000), temperature=21.067142486572266)
Row(warehouse_id=UUID('8bfed691-e001-402c-97aa-a1e972812ccb'), timestamp=datetime.datetime(2023, 8, 13, 10, 57, 7, 978000), temperature=21.609975814819336)
Row(warehouse_id=UUID('8bfed691-e001-402c-97aa-a1e972812ccb'), timestamp=datetime.datetime(2023, 8, 13, 10, 57, 7, 963000), temperature=16.166860580444336)
Row(warehouse_id=UUID('8bfed691-e001-402c-97aa-a1e972812ccb'), timestamp=datetime.datetime(2023, 8, 13, 10, 57, 7, 948000), temperature=24.03549575805664)
Row(warehouse_id=UUID('8bfed691-e001-402c-97aa-a1e972812ccb'), timestamp=datetime.datetime(2023, 8, 13, 10, 57, 7, 932000), temperature=21.299055099487305)
Row(warehouse_id=UUID('8bfed691-e001-402c-97aa-a1e972812ccb'), time

In [395]:
# Neo4
# Connecting to Neo4

In [396]:
from neo4j import GraphDatabase

In [397]:
uri = "neo4j+s://170b1206.databases.neo4j.io"
username = "neo4j"
password = "4JPi8xWrpKhlrAslT-XKAFqiIZsFLurn-WHLq-sr58Q"

In [398]:
driver = GraphDatabase.driver(uri,auth=(username,password))

In [399]:
# Graph Creation:
# Build a graph database to represent a social network, with nodes for users and relationships for friendships.
# Create nodes for users and relationships to model their interactions.

In [469]:
def create_graph(tx):
    tx.run("CREATE (alice:User {name: 'Alice'})"
           "CREATE (carl:User {name: 'Carl'})"
           "CREATE (sandra:User {name: 'Sandra'})"
           "CREATE (christine:User {name: 'Christine'})"
           "CREATE (item1:Item {name: 'Movie'})"
           "CREATE (item2:Item {name: 'Book'})"
           "CREATE (alice)-[:FRIEND]->(carl)"
           "CREATE (carl)-[:FRIEND]->(christine)"
           "CREATE (sandra)-[:FRIEND]->(carl)"
           "CREATE (carl)-[:FRIEND]->(sandra)"
           "CREATE (alice)-[:LIKES]->(item1)"
           "CREATE (carl)-[:LIKES]->(item1)"
           "CREATE (sandra)-[:LIKES]->(item2)"
           "CREATE (christine)-[:LIKES]->(item2)"
           "CREATE (alice)-[:LIKES]->(item2)"
           "CREATE (carl)-[:FRIEND]->(alice)"
           "CREATE (sandra)-[:FRIEND]->(alice)"
           "CREATE (christine)-[:FRIEND]->(carl)")

In [401]:
# Querying Graph Data:
# Write a Cypher query to find all friends of a given user up to a certain depth (degree of separation).
# Retrieve common friends between two users in the graph.

In [414]:
def find_friends(tx, user_name, depth):
    query = (
        "MATCH (u:User {name: $user_name})-[:FRIEND*1.." + str(depth) + "]->(f) "
        "RETURN DISTINCT f.name"
    )
    result = tx.run(query, user_name=user_name)
    return [record["f.name"] for record in result]


def find_common_friends(tx, user1_name, user2_name):
    result = tx.run("MATCH (u1:User {name: $user1_name})-[:FRIEND]->(f:User)<-[:FRIEND]-(u2:User {name: $user2_name})"
                    "RETURN DISTINCT f.name", user1_name=user1_name, user2_name=user2_name)
    return [record["f.name"] for record in result]


In [409]:
# Recommendation System:
# Implement a basic recommendation system by finding users who have similar interests based on their interactions.
# Propose movies, books, or other items that a user might be interested in based on their connections' preferences.

In [416]:
def recommend_similar_users(tx, user_name):
    result = tx.run("MATCH (u1:User {name: $user_name})-[:FRIEND]->(:User)-[:FRIEND]->(u2:User)"
                    "RETURN DISTINCT u2.name", user_name=user_name)
    return [record["u2.name"] for record in result]

def recommend_interest_based(tx, user_name):
    result = tx.run("MATCH (u1:User {name: $user_name})-[:FRIEND]->(f:User)-[:LIKES]->(item)"
                    "RETURN DISTINCT item.name", user_name=user_name)
    return [record["item.name"] for record in result]

In [411]:
# Pathfinding:
# Use graph algorithms in Neo4j to find the shortest path between two nodes in the graph.
# Find the path of mutual friends between two users.

In [497]:
def find_shortest_path_direct_friends(tx, start_name, end_name):
    result = tx.run(
        "MATCH p=allShortestPaths((start:User {name: $start_name})-[:FRIEND*1..3]-(end:User {name: $end_name})) "
        "RETURN p", start_name=start_name, end_name=end_name)
    
    processed_paths_set = set()
    
    for record in result:
        path = record["p"]
        processed_path = [node["name"] for node in path.nodes]
        processed_paths_set.add(tuple(processed_path))
    
    return [list(path) for path in processed_paths_set]


def find_mutual_friends_path(tx, user1_name, user2_name):
    result = tx.run("MATCH (u1:User {name: $user1_name})-[:FRIEND]-(mf:User)-[:FRIEND]-(u2:User {name: $user2_name})"
                    "RETURN COLLECT(DISTINCT mf.name) AS mutual_friends", user1_name=user1_name, user2_name=user2_name)
    record = result.single()
    if record:
        return record["mutual_friends"]
    return None


In [499]:
# Execute data creation
with driver.session() as session:
    session.write_transaction(create_graph)
    print(find_friends(session, "Alice", 1))
    print(find_common_friends(session, "Alice", "Sandra"))
    print(recommend_similar_users(session, "Alice"))
    print(recommend_interest_based(session, "Alice"))
    print(find_mutual_friends_path(session, "Alice", "Sandra"))
    print(find_shortest_path_direct_friends(session, "Alice", "Christine"))




C:\Users\37129\AppData\Local\Temp\ipykernel_14396\2536233134.py:3: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_graph)


['Christine', 'Sandra', 'Carl']
['Carl']
['Carl', 'Alice', 'Sandra', 'Christine']
['Book', 'Movie']
['Carl']
[['Alice', 'Carl', 'Christine'], ['Alice', 'Christine']]
